# DIRECT INFERENCING 

In [1]:
import os
import random
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


class SpeechFeatureEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

In [3]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [4]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm

In [5]:
class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_input = SpeechFeatureEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

In [6]:
class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 2303) for i in range(1, 120)] 
#             + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )
        
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i
            print(self.char_to_idx[ch],ch)

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
#         print(self.vocab)
        return self.vocab
        


max_target_len = 200  # all transcripts in our data are < 200 characters
# data = get_data(wavs, id_to_text, max_target_len)
# # print(data)
vectorizer = VectorizeChar(max_target_len)
# print("vocab size", len(vectorizer.get_vocabulary()))

# import random
# random.shuffle(data)


def create_text_ds(data):
    texts = [_["text"] for _ in data]
    text_ds = [vectorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
#     print(text_ds)
    return text_ds


def path_to_audio(path):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)

    x = tf.math.pow(tf.abs(stfts), 0.5)

    # normalisation
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = (x - means) / stddevs
    audio_len = tf.shape(x)[0]

    # padding to 10 seconds
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    x = tf.where(tf.math.is_nan(x), 0., x) # get rid of all nan values, avoid "loss:nan"

    return x


def create_audio_ds(data):
    flist = [_["audio"] for _ in data]
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(
        path_to_audio, num_parallel_calls=tf.data.AUTOTUNE
    )
    return audio_ds


def create_tf_dataset(data, bs=4):
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


# split = int(len(data) * 0.99) #spliting total 157905 data into 99:1. or 
# train_data = data[:split] #99% of total data ie: 156325
# print(train_data[0])
# test_data = data[split:]  #1580
# # print(test_data[0])
# ds = create_tf_dataset(train_data, bs=64) #2443 ie: total train_data divided by batch size.
# # print(ds)
# val_ds = create_tf_dataset(test_data, bs=4) #395 ie: total test_data divided by batch size 4
# print("the tensor of validaiton dataset:\n")
# # print(val_ds)#tensor structure of source and target data


# #Spliting Train and Val+Train data
# split = int(len(data) * 0.96)
# train_data = data[:split]
# TestnVal_data = data[split:]
# #Now Splitting test and validation data From Val+Train
# split2 = int(len(TestnVal_data)*0.6)
# test_data = TestnVal_data[:split2]
# FinalVal_data = TestnVal_data[split2:]
# ds = create_tf_dataset(train_data, bs=64)
# val_ds = create_tf_dataset(test_data, bs=4)

0 -
1 #
2 <
3 >
4 ऀ
5 ँ
6 ं
7 ः
8 ऄ
9 अ
10 आ
11 इ
12 ई
13 उ
14 ऊ
15 ऋ
16 ऌ
17 ऍ
18 ऎ
19 ए
20 ऐ
21 ऑ
22 ऒ
23 ओ
24 औ
25 क
26 ख
27 ग
28 घ
29 ङ
30 च
31 छ
32 ज
33 झ
34 ञ
35 ट
36 ठ
37 ड
38 ढ
39 ण
40 त
41 थ
42 द
43 ध
44 न
45 ऩ
46 प
47 फ
48 ब
49 भ
50 म
51 य
52 र
53 ऱ
54 ल
55 ळ
56 ऴ
57 व
58 श
59 ष
60 स
61 ह
62 ऺ
63 ऻ
64 ़
65 ऽ
66 ा
67 ि
68 ी
69 ु
70 ू
71 ृ
72 ॄ
73 ॅ
74 ॆ
75 े
76 ै
77 ॉ
78 ॊ
79 ो
80 ौ
81 ्
82 ॎ
83 ॏ
84 ॐ
85 ॑
86 ॒
87 ॓
88 ॔
89 ॕ
90 ॖ
91 ॗ
92 क़
93 ख़
94 ग़
95 ज़
96 ड़
97 ढ़
98 फ़
99 य़
100 ॠ
101 ॡ
102 ॢ
103 ॣ
104 ।
105 ॥
106 ०
107 १
108 २
109 ३
110 ४
111 ५
112 ६
113 ७
114 ८
115 ९
116 ॰
117 ॱ
118 ॲ
119 ॳ
120 ॴ
121 ॵ
122 ॶ
123  
124 .
125 ,
126 ?


In [7]:
class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Displays a batch of outputs after every epoch

        Args:
            batch: A test batch containing the keys "source" and "target"
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        print(batch)
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")

In [8]:
model = Transformer(
    num_hid=200,
    num_head=2,
    num_feed_forward=400,
    target_maxlen=max_target_len,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=127,
)

In [25]:
model_path = r"D:\\shishir_ml\\Untitled Folder\\TestModeltwo"
model.load_weights(model_path)

In [23]:
model_path = r"D:\meroDs\MyNewAudioDatasetsTHESIS\MyFinalDataset\TrainedModels\myDatasetCheckpoint-30-0.31"
model.load_weights(model_path)

In [11]:
model_path = r"F:\\TRAINEDModelSfromActiveLearningLabSharing\\Untitled Folder\\ModelFor99isTo1\\Checkpoints for 99 is to 1\\smallDatasetCheckpoint-57-0.12"
model.load_weights(model_path)

In [13]:
# this model is fine
model_path = r"F:\\TRAINEDModelSfromActiveLearningLabSharing\\Untitled Folder\\ModelFor99isTo1\\Checkpoints for 99 is to 1\\smallDatasetCheckpoint-56-0.12"
model.load_weights(model_path)

In [9]:
# New 2023 Model
# model_path = r"F:\\TRAINEDModelSfromActiveLearningLabSharing\\Untitled Folder\\ModelFor99isTo1\\Checkpoints for 99 is to 1\\smallDatasetCheckpoint-48-0.12"
# model.load_weights(model_path)

model_path = r"D:\New folder\TOBECOPIEDTONEWLAPTOP\THESIS Important\THESIS CODING\ActivelearningLabsharing\bestMODELS\smallDatasetCheckpoint-48-0.12"
model.load_weights(model_path)


In [10]:
#PREDICTING ANY AUDIO
print("Loading the transcription")
path="D:\\shishir_ml\\Untitled Folder/Rec/Audiotest.wav"


# print(path)

x = path_to_audio(path)
#print(x)
x = tf.expand_dims(x, axis=0)
# print(x.shape)
idx_to_char = vectorizer.get_vocabulary()
preds = model.generate(x, 2)
preds = preds.numpy()
bs = tf.shape(x)[0]
for i in range(bs):
    prediction = ""
    for idx in preds[i, :]:
        prediction += idx_to_char[idx]
        if idx == 3:
            break
            
print("prediction: ", prediction)


Loading the transcription


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: D:\shishir_ml\Untitled Folder/Rec/Audiotest.wav : The system cannot find the path specified.
; No such process [Op:ReadFile]

# SKIP FROM HERE

# OLD CER CALCULATION

In [11]:

def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy

    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]


if __name__ == "__main__":
    import doctest

    doctest.testmod()

In [ ]:
#PREDICTING FROM TEST DATA Alredy downsampled
# PREDICTION and WER EVALUATION FOR ANYDATA.
myfile = open('D:\\shishir_ml\\Untitled Folder\datasets\LJSpeech\\utt_spk_text.tsv', 'r',encoding = 'utf-8')
Lines = myfile.readlines()

count = 0
my_list=[]
# Strips the newline character
for line in Lines:
    count += 1
    # wordarr = line.split()
    arr = line.split("\t")
    my_list.append(arr)
    # print (arr)       
# audio_iddd=arr[0]
# audio_text=arr[2]
    # import required module
from playsound import playsound

# path="D:\\shishir_ml\\Untitled Folder\\tess_1.wav"

wer_list=[]
for xz in range(157725,157730):
# for xz in range(1):

    path="D:\\shishir_ml\\Untitled Folder\datasets\LJSpeech/wavs/" + my_list[xz][0]+".wav" #the audio


    # for playing note.wav file
    print("Playing the audio file")
#     playsound(path)
    
    referencetxtt=my_list[xz][2]
    x = path_to_audio(path)
    #print(x)
    x = tf.expand_dims(x, axis=0)
    # print(x.shape)
    idx_to_char = vectorizer.get_vocabulary()
    preds = model.generate(x, 2)
    preds = preds.numpy()
    bs = tf.shape(x)[0]
    for i in range(bs):
        prediction = ""
        for idx in preds[i, :]:
            prediction += idx_to_char[idx]
            if idx == 3:
                break
#     print("reference:", referencetxtt)
#     print("prediction: ", prediction,"\n")
    
    ref= referencetxtt
    hyp= prediction
    new=ref.replace("\n", "")
    new_ref="<"+new+">"
    print("Reference:",new_ref)
    print("Prediction:",prediction)
    werr=wer(new_ref,hyp)
    print("CER:")
    print(werr)
    wer_list.append(werr)
    
    
    
print(wer_list)
print("the avg cer is:")
Avg_wer=sum(wer_list) / len(wer_list)
print(Avg_wer)

In [13]:
#PREDICING FOR NON- DOWNSAMPLED DATASET
##Audio TRANSLATION
# import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
import sounddevice

In [25]:
#TESTING FROM ANOTHER DATASETS ( NEEDS MORE TRAINING EPOCHS BEFORE GOOD DEMO)
myfile = open(r'D:\shishir_ml\Untitled Folder\my_Test_datasetss\line_index.tsv', 'r',encoding = 'utf-8')
Lines = myfile.readlines()
from pydub import AudioSegment as am

count = 0
my_lisst=[]
# Strips the newline character
for line in Lines:
    count += 1
    # wordarr = line.split()
    arr = line.split("\t")
    my_lisst.append(arr)
    # print (arr)       
# audio_iddd=arr[0]
# audio_text=arr[2]
    # import required module

from playsound import playsound

# path="D:\\shishir_ml\\Untitled Folder\\tess_1.wav"

weer_list=[]
# for xxz in range(2063):
for xxz in range(10):

    filepath= r"D:\shishir_ml\Untitled Folder\my_Test_datasetss\wavs/" + my_lisst[xxz][0]+".wav" #the audio
#     print("Playing the audio file...")
#     playsound(filepath)
    saving_dir=f"D:\\shishir_ml\\Untitled Folder\\my_Test_datasets\\Converted\\{xxz}.wav"
    sound = am.from_file(filepath, format='wav')
    sound = sound.set_frame_rate(16000)
    sound.export(saving_dir, format='wav')
    pathh= saving_dir
    # for playing note.wav file
 

    referencetxttz=my_lisst[xxz][1]
    x = path_to_audio(pathh)
    #print(x)
    x = tf.expand_dims(x, axis=0)
    # print(x.shape)
    idx_to_char = vectorizer.get_vocabulary()
    preds = model.generate(x, 2)
    preds = preds.numpy()
    bs = tf.shape(x)[0]
    for i in range(bs):
        prediction = ""
        for idx in preds[i, :]:
            prediction += idx_to_char[idx]
            if idx == 3:
                break
#     print("reference:", referencetxtt)
#     print("prediction: ", prediction,"\n")
    
    ref= referencetxttz
    hyp= prediction
    new=ref.replace("\n", "")
    new_ref="<"+new+">"
    print("Reference:",new_ref)
    print("Prediction:",prediction)
    
    print(new_ref.split(),hyp.split())
    totError=wer(new_ref,hyp)
    totworderror=wer(new_ref.split(),hyp.split())
    print("totalCharError:",totError)
    print("totalWordError:",totworderror)
    weerr=(totError/len(new_ref))*100
    wwerrr=(totworderror/len(new_ref.split()))*100
    
#     weerr=wer(new_ref,hyp) 
    print("Sen_no:",xxz)
    print("CER:",weerr,"%")
    print("WER:",wwerrr,"%")
    weer_list.append(weerr)
    
print(weer_list)
print("the avg cer is:")
Avg_wer=sum(weer_list) / len(weer_list)
print(Avg_wer)

Reference: <दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्लामा भएको हो>
Prediction: <दिपादन सुदूरमा सुदूर्व स्वीलामा भएको हो>
['<दीपा', 'धामीको', 'जन्म', 'सुदूरपश्चिम', 'नेपालको', 'बझाङ', 'जिल्लामा', 'भएको', 'हो>'] ['<दिपादन', 'सुदूरमा', 'सुदूर्व', 'स्वीलामा', 'भएको', 'हो>']
OP	REF	HYP
DEL	<दीपा	****
DEL	धामीको	****
DEL	जन्म	****
SUB	सुदूरपश्चिम	<दिपादन
SUB	नेपालको	सुदूरमा
SUB	बझाङ	सुदूर्व
SUB	जिल्लामा	स्वीलामा
OK	भएको	भएको
OK	हो>	हो>
#cor 2
#sub 4
#del 3
#ins 0
wer: 0.778


AttributeError: 'list' object has no attribute 'split'

# CONTInue FROM HERE

# ###LIVE AUDIO####

In [12]:
!pip install sounddevice


                                              0.0/199.7 kB ? eta -:--:--
     --                                       10.2/199.7 kB ? eta -:--:--
     -----                                 30.7/199.7 kB 325.1 kB/s eta 0:00:01
     -----                                 30.7/199.7 kB 325.1 kB/s eta 0:00:01
     -----                                 30.7/199.7 kB 325.1 kB/s eta 0:00:01
     -----                                 30.7/199.7 kB 325.1 kB/s eta 0:00:01
     -----                                 30.7/199.7 kB 325.1 kB/s eta 0:00:01
     -----------                           61.4/199.7 kB 181.6 kB/s eta 0:00:01
     -----------------                     92.2/199.7 kB 238.1 kB/s eta 0:00:01
     --------------------                 112.6/199.7 kB 273.1 kB/s eta 0:00:01
     --------------------                 112.6/199.7 kB 273.1 kB/s eta 0:00:01
     --------------------                 112.6/199.7 kB 273.1 kB/s eta 0:00:01
     ----------------------               122.9/199.7


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
##LIVE RECORD TRANSLATION (FOR DEMO)
# import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
#animation imports
import itertools
import threading
import time
import sys
import sounddevice

In [11]:
done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rRecording ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!')

In [33]:
fs= 16000  
second = 4

t = threading.Thread(target=animate)

t.start()



#long process here
print("Please Speak in Nepali")
record_voice = sounddevice.rec( int ( second * fs ) , samplerate = fs , channels = 1, dtype='int16' )
sounddevice.wait()

time.sleep(1)
done = True

# fs= 16000
# # second =  int(input("Enter time duration in seconds: "))
# second = 8
# print("Please speak in Nepali")
# print("Recording.....\n")
# record_voice = sounddevice.rec( int ( second * fs ) , samplerate = fs , channels = 1, dtype='int16' )
# sounddevice.wait()
aud_path=r"D:\New folder\TOBECOPIEDTONEWLAPTOP\THESIS Important\THESIS CODING\ActivelearningLabsharing\bestMODELS\Audio_liveRecords/Audiotest.wav"
write(aud_path,fs,record_voice)
print("Saved in REC folder")

Done!Please Speak in Nepali
Saved in REC folder


In [14]:
# #Playing the recorded audio
# # import required module
# from playsound import playsound
# # for playing note.wav file
# print("Playing the recorded audio")
# playsound(aud_path)

Playing the recorded audio


In [34]:
#Prediction of the recorded audio.
print("Loading the transcription")
path=r"D:\New folder\TOBECOPIEDTONEWLAPTOP\THESIS Important\THESIS CODING\ActivelearningLabsharing\bestMODELS\Audio_liveRecords/Audiotest.wav"

# print(path)

x = path_to_audio(path)
#print(x)
x = tf.expand_dims(x, axis=0)
# print(x.shape)
idx_to_char = vectorizer.get_vocabulary()
preds = model.generate(x, 2)
preds = preds.numpy()
bs = tf.shape(x)[0]
for i in range(bs):
    prediction = ""
    for idx in preds[i, :]:
        prediction += idx_to_char[idx]
        if idx == 3:
            break
            
print("prediction: ", prediction)


Loading the transcription
prediction:  <सरलाई बोल्दै बोल्दैन>


In [17]:
model_path = r"F:\\TRAINEDModelSfromActiveLearningLabSharing\\Untitled Folder\\ModelFor99isTo1\\Checkpoints for 99 is to 1\\smallDatasetCheckpoint-56-0.12"
model.load_weights(model_path)

In [ ]:
# Unitl here for demo


# New WER calcualtion

In [18]:
# link for this: https://pyzone.dev/word-error-rate-in-python/

weer_list=[]
def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    # return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    print("wer:",wer_result)
    weer_list.append(wer_result)
    return {'WER':wer_result, 'numCor':numCor, 'numSub':numSub, 'numIns':numIns, 'numDel':numDel, "numCount": len(r)}
# Avg_wer=sum(weer_list) / len(weer_list)
# print(Avg_wer)

## 1. For 2064 dataset

In [31]:
#TESTING FROM ANOTHER DATASETS ( NEEDS MORE TRAINING EPOCHS BEFORE GOOD DEMO)
myfile = open(r'D:\shishir_ml\Untitled Folder\my_Test_datasetss\line_index.tsv', 'r',encoding = 'utf-8')
Lines = myfile.readlines()
from pydub import AudioSegment as am

count = 0
my_lisst=[]
# Strips the newline character
for line in Lines:
    count += 1
    # wordarr = line.split()
    arr = line.split("\t")
    my_lisst.append(arr)
    # print (arr)       
# audio_iddd=arr[0]
# audio_text=arr[2]
    # import required module

from playsound import playsound

# path="D:\\shishir_ml\\Untitled Folder\\tess_1.wav"

weer_list=[]
# for xxz in range(2063):
for xxz in range(150,153):

    filepath= r"D:\shishir_ml\Untitled Folder\my_Test_datasetss\wavs/" + my_lisst[xxz][0]+".wav" #the audio
#     print("Playing the audio file...")
#     playsound(filepath)
    saving_dir=f"D:\\shishir_ml\\Untitled Folder\\my_Test_datasets\\Converted\\{xxz}.wav"
    sound = am.from_file(filepath, format='wav')
    sound = sound.set_frame_rate(16000)
    sound.export(saving_dir, format='wav')
    pathh= saving_dir
    # for playing note.wav file
 

    referencetxttz=my_lisst[xxz][1]
    x = path_to_audio(pathh)
    #print(x)
    x = tf.expand_dims(x, axis=0)
    # print(x.shape)
    idx_to_char = vectorizer.get_vocabulary()
    preds = model.generate(x, 2)
    preds = preds.numpy()
    bs = tf.shape(x)[0]
    for i in range(bs):
        prediction = ""
        for idx in preds[i, :]:
            prediction += idx_to_char[idx]
            if idx == 3:
                break
#     print("reference:", referencetxtt)
#     print("prediction: ", prediction,"\n")
    

    new=referencetxttz.replace("\n", "")
    new_ref="<"+new+">"
    print("Reference:",new_ref)
    print("Prediction:",prediction)
    
#     print(new_ref.split(),hyp.split())
#     totError=wer(new_ref,hyp)
#     totworderror=wer(new_ref.split(),hyp.split())
#     print("totalCharError:",totError)
#     print("totalWordError:",totworderror)
#     weerr=(totError/len(new_ref))*100
#     wwerrr=(totworderror/len(new_ref.split()))*100
#     weerr=wer(new_ref,hyp) 

    print("Sen_no:",xxz)
    ref=new_ref
    hyp=prediction
    wer(ref,hyp)
    print(weer_list)
    
    
print(weer_list)
print("the avg wer is:")
Avg_wer=sum(weer_list) / len(weer_list)
print(Avg_wer)

Reference: <रुसको सङ्घीय विषय रुसको क्राइ>
Prediction: <रुसको सङ्ख्य विषय रोसको क्राइ>
Sen_no: 150
OP	REF	HYP
OK	<रुसको	<रुसको
SUB	सङ्घीय	सङ्ख्य
OK	विषय	विषय
SUB	रुसको	रोसको
OK	क्राइ>	क्राइ>
#cor 3
#sub 2
#del 0
#ins 0
wer: 0.4
[0.4]
Reference: <बाइनरी ताराहरूको वर्ग>
Prediction: <बाइनरी ताराहरूको वर्ग>
Sen_no: 151
OP	REF	HYP
OK	<बाइनरी	<बाइनरी
OK	ताराहरूको	ताराहरूको
OK	वर्ग>	वर्ग>
#cor 3
#sub 0
#del 0
#ins 0
wer: 0.0
[0.4, 0.0]
Reference: <उन्नाइस सय छयानब्बे र सन्तानब्बेमा कोलम्बियाली फुटबल व्यावसायिकको पहिलो श्रेणी>
Prediction: <उन्नाइस सय छाया नबेर सन्तानबेमा कोलम्बेर साएको पहिलो स्वर्ग व्यवसायिको पहिलो स्रेनी>
Sen_no: 152
OP	REF	HYP
OK	<उन्नाइस	<उन्नाइस
OK	सय	सय
INS	****	छाया
INS	****	नबेर
SUB	छयानब्बे	सन्तानबेमा
SUB	र	कोलम्बेर
SUB	सन्तानब्बेमा	साएको
SUB	कोलम्बियाली	पहिलो
SUB	फुटबल	स्वर्ग
SUB	व्यावसायिकको	व्यवसायिको
OK	पहिलो	पहिलो
SUB	श्रेणी>	स्रेनी>
#cor 3
#sub 7
#del 0
#ins 2
wer: 0.9
[0.4, 0.0, 0.9]
[0.4, 0.0, 0.9]
the avg cer is:
0.43333333333333335


In [32]:
print("total no of WER values:",len(weer_list))
Avg_WER=sum(weer_list)/len(weer_list)
print(Avg_WER)

total no of WER values: 3
0.43333333333333335


## 2. For 166K dataset

In [19]:
#PREDICTING FROM TEST DATA Alredy downsampled
# PREDICTION and WER EVALUATION FOR ANYDATA.
myfile = open('D:\\shishir_ml\\Untitled Folder\datasets\LJSpeech\\utt_spk_text.tsv', 'r',encoding = 'utf-8')
Lines = myfile.readlines()

count = 0
my_list=[]
# Strips the newline character
for line in Lines:
    count += 1
    # wordarr = line.split()
    arr = line.split("\t")
    my_list.append(arr)
    # print (arr)       
# audio_iddd=arr[0]
# audio_text=arr[2]
    # import required module
from playsound import playsound

# path="D:\\shishir_ml\\Untitled Folder\\tess_1.wav"

weer_list=[]
for xz in range(157725,157730):
# for xz in range(1):

    path="D:\\shishir_ml\\Untitled Folder\datasets\LJSpeech/wavs/" + my_list[xz][0]+".wav" #the audio


    # for playing note.wav file
#     print("Playing the audio file")
#     playsound(path)
    
    referencetxtt=my_list[xz][2]
    x = path_to_audio(path)
    #print(x)
    x = tf.expand_dims(x, axis=0)
    # print(x.shape)
    idx_to_char = vectorizer.get_vocabulary()
    preds = model.generate(x, 2)
    preds = preds.numpy()
    bs = tf.shape(x)[0]
    for i in range(bs):
        prediction = ""
        for idx in preds[i, :]:
            prediction += idx_to_char[idx]
            if idx == 3:
                break
#     print("reference:", referencetxtt)
#     print("prediction: ", prediction,"\n") prediction,"\n")
    

    new=referencetxtt.replace("\n", "")
    new_ref="<"+new+">"
    print("Reference:",new_ref)
    print("Prediction:",prediction)
    
#     print(new_ref.split(),hyp.split())
#     totError=wer(new_ref,hyp)
#     totworderror=wer(new_ref.split(),hyp.split())
#     print("totalCharError:",totError)
#     print("totalWordError:",totworderror)
#     weerr=(totError/len(new_ref))*100
#     wwerrr=(totworderror/len(new_ref.split()))*100
#     weerr=wer(new_ref,hyp) 

    print("Sen_no:",xz)
    ref=new_ref
    hyp=prediction
    wer(ref,hyp)
    print(weer_list)
    
    
print(weer_list)
print("the avg wer is:")
Avg_wer=sum(weer_list) / len(weer_list)
print(Avg_wer)

Reference: <हो भने नदी>
Prediction: <हो भनिन् ती>
Sen_no: 157725
OP	REF	HYP
OK	<हो	<हो
SUB	भने	भनिन्
SUB	नदी>	ती>
#cor 1
#sub 2
#del 0
#ins 0
wer: 0.667
[0.667]
Reference: <हो भने नदी>
Prediction: <हो भने नदी>
Sen_no: 157726
OP	REF	HYP
OK	<हो	<हो
OK	भने	भने
OK	नदी>	नदी>
#cor 3
#sub 0
#del 0
#ins 0
wer: 0.0
[0.667, 0.0]
Reference: <हो भने निर्माणस्थलमा>
Prediction: <हो भने निर्माण स्थलमा>
Sen_no: 157727
OP	REF	HYP
OK	<हो	<हो
OK	भने	भने
INS	****	निर्माण
SUB	निर्माणस्थलमा>	स्थलमा>
#cor 2
#sub 1
#del 0
#ins 1
wer: 0.667
[0.667, 0.0, 0.667]
Reference: <हो भने निर्माणस्थलमा>
Prediction: <भूभने निर्माण स्तरमा>
Sen_no: 157728
OP	REF	HYP
SUB	<हो	<भूभने
SUB	भने	निर्माण
SUB	निर्माणस्थलमा>	स्तरमा>
#cor 0
#sub 3
#del 0
#ins 0
wer: 1.0
[0.667, 0.0, 0.667, 1.0]
Reference: <हो भने निर्माणस्थलमा>
Prediction: <कोबने निर्माणिस्तान>
Sen_no: 157729
OP	REF	HYP
DEL	<हो	****
SUB	भने	<कोबने
SUB	निर्माणस्थलमा>	निर्माणिस्तान>
#cor 0
#sub 2
#del 1
#ins 0
wer: 1.0
[0.667, 0.0, 0.667, 1.0, 1.0]
[0.667, 0.0, 0.667, 

In [34]:
print("total no of WER values:",len(weer_list))
Avg_WER=sum(weer_list)/len(weer_list)
print(Avg_WER)

total no of WER values: 5
0.5334


In [2]:
ne_np_female=[0.4, 0.0, 1.1, 0.571, 0.111, 0.571, 0.167, 0.583, 0.333, 0.545, 0.769, 0.6, 1.0, 0.667, 0.333, 0.0, 0.5, 0.556, 0.333, 0.769, 0.615, 0.2, 0.333, 0.556, 0.75, 0.111, 0.4, 0.429, 0.0, 0.5, 0.778, 1.182, 0.688, 0.75, 0.778, 0.75, 0.556, 1.0, 0.5, 0.778, 1.083, 0.429, 0.6, 0.455, 0.909, 0.333, 1.0, 0.5, 0.5, 0.286, 0.286, 0.636, 0.091, 1.0, 0.588, 0.9, 0.6, 0.882, 0.667, 0.667, 0.2, 0.444, 0.6, 1.0, 0.727, 0.85, 0.667, 0.5, 0.25, 1.0, 0.667, 0.455, 0.25, 0.6, 0.143, 0.375, 0.636, 0.444, 0.222, 0.444, 0.944, 0.75, 0.667, 0.545, 0.8, 0.5, 0.667, 0.778, 0.75, 0.667, 0.778, 0.444, 0.786, 0.4, 0.2, 0.714, 0.5, 0.583, 0.643, 0.846, 0.167, 0.667, 0.222, 0.556, 0.7, 0.7, 0.8, 0.933, 0.667, 0.875, 0.75, 0.5, 0.75, 0.556, 1.5, 0.7, 0.333, 0.375, 0.6, 0.556, 0.8, 0.333, 0.75, 0.6, 0.333, 0.0, 0.333, 0.4, 0.267, 0.0, 0.895, 1.6, 0.5, 0.333, 0.4, 0.75, 1.364, 1.0, 0.538, 0.0, 0.2, 0.4, 0.769, 0.5, 0.429, 0.667, 0.2, 0.25, 0.444, 0.6, 0.778, 0.0, 0.429, 1.2, 0.25, 0.0, 0.2, 0.111, 0.083, 0.444, 0.267, 0.545, 0.3, 0.333, 0.444, 0.5, 0.333, 0.333, 0.9, 0.5, 0.375, 0.0, 0.842, 0.692, 0.444, 0.75, 0.571, 1.0, 0.429, 0.667, 0.333, 1.0, 0.222, 0.429, 1.0, 0.5, 0.4, 1.0, 0.462, 0.0, 0.385, 0.167, 0.688, 0.0, 0.385, 0.0, 0.375, 0.167, 0.625, 0.75, 0.667, 0.6, 0.444, 0.333, 0.333, 0.6, 0.4, 0.2, 0.4, 0.333]

merged_all=[0.333, 0.0, 1.0, 0.0, 0.667, 0.5, 1.5, 1.5, 0.0, 1.0, 1.0, 0.667, 0.667, 0.0, 1.0, 0.667, 0.667, 0.667, 0.0, 0.0, 0.333, 1.0, 0.667, 0.5, 0.75, 0.0, 0.667, 0.667, 0.0, 1.0, 1.0, 0.667, 0.0, 0.0, 0.667, 0.333, 0.333, 1.0, 0.0, 0.0, 0.333, 0.75, 0.333, 0.0, 0.75, 0.5, 1.273, 0.333, 0.0, 0.5, 1.0, 1.0, 0.333, 0.0, 0.5, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 1.0, 0.0, 0.333, 0.333, 0.333, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 1.0, 1.0, 0.667, 0.667, 0.667, 0.0, 0.333, 0.333, 0.333, 0.667, 0.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.5, 1.5, 0.5, 1.0, 1.625, 1.667, 0.0, 2.0, 1.0, 0.5, 2.0, 0.5, 0.333, 0.0, 0.667, 0.0, 0.667, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.5, 0.667, 0.333, 0.667, 0.5, 0.5, 0.5, 1.5, 1.0, 0.667, 1.333, 0.333, 1.0, 0.667, 1.5, 1.0, 0.667, 0.333, 1.0, 0.333, 0.333, 1.0, 0.333, 0.333, 0.667, 0.0, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 1.333, 1.0, 0.667, 0.333, 0.333, 0.5, 0.667, 1.0, 1.0, 0.0, 0.667, 0.4, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.5, 0.0, 0.5, 0.0, 0.0, 0.667, 1.0, 0.0, 0.5, 0.5, 0.8, 0.4, 0.667, 0.0, 0.0, 0.0, 0.667, 0.333, 1.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.667, 1.0, 1.0, 1.0, 0.5, 0.667, 1.0, 0.333, 0.5, 1.5, 1.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.5, 1.0, 0.0, 1.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.0, 0.667, 0.667, 0.333, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.333, 0.0, 0.667, 0.333, 1.0, 0.0, 0.333, 0.667, 0.667, 0.333, 1.0, 0.667, 0.667, 0.667, 0.5, 1.5, 1.5, 0.5, 0.5, 1.0, 1.5, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.333, 1.0, 0.333, 0.5, 0.5, 0.333, 0.625, 0.875, 0.8, 0.667, 0.667, 0.667, 1.0, 0.25, 0.75, 1.0, 2.0, 2.333, 1.0, 1.0, 0.667, 0.667, 0.333, 0.0, 1.0, 0.667, 0.667, 0.0, 0.727, 0.667, 0.667, 0.667, 1.0, 0.333, 0.333, 0.667, 1.0, 1.0, 1.0, 1.333, 2.667, 0.0, 1.5, 2.0, 0.5, 0.5, 0.0, 0.333, 0.667, 1.0, 0.667, 0.667, 0.5, 0.0, 0.5, 0.0, 0.5, 1.0, 0.0, 1.0, 1.0, 0.333, 0.0, 0.333, 2.0, 2.0, 0.333, 0.667, 0.333, 0.0, 0.667, 1.0, 0.0, 0.333, 0.333, 1.0, 0.333, 0.667, 0.0, 1.0, 0.333, 0.333, 0.667, 1.667, 0.0, 0.0, 0.0, 0.333, 1.0, 0.667, 0.667, 0.333, 0.667, 1.0, 1.0, 0.333, 1.0, 0.667, 1.0, 0.0, 0.667, 0.625, 1.0, 0.667, 0.0, 0.667, 0.667, 0.667, 0.222, 0.444, 1.0, 0.5, 0.333, 0.0, 0.0, 2.0, 1.0, 0.667, 1.0, 1.333, 1.333, 1.0, 0.667, 0.0, 2.0, 0.667, 1.0, 0.0, 0.0, 0.0, 0.667, 0.333, 1.0, 0.667, 1.0, 0.333, 0.667, 0.333, 0.333, 0.333, 0.333, 0.0, 1.0, 0.0, 1.0, 1.667, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.333, 1.333, 1.333, 0.0, 0.667, 0.667, 0.667, 0.667, 0.0, 0.333, 0.0, 0.667, 0.333, 0.667, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 0.0, 0.333, 0.333, 1.0, 1.0, 0.333, 0.667, 0.667, 0.846, 1.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.333, 0.0, 0.5, 0.5, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 1.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.333, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.667, 0.333, 0.333, 0.333, 0.333, 0.0, 1.0, 0.5, 0.5, 1.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.333, 0.333, 0.4, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.5, 0.0, 0.0, 0.333, 1.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.5, 0.333, 0.0, 0.333, 0.333, 0.333, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 0.5, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 1.0, 1.0, 0.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.0, 0.667, 0.0, 0.333, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 1.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.667, 0.667, 0.333, 1.0, 0.333, 0.667, 0.5, 0.0, 0.333, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.5, 0.5, 0.0, 1.0, 0.0, 1.0, 0.0, 0.5, 0.5, 1.0, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 1.0, 1.0, 0.667, 1.0, 1.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 1.0, 0.75, 0.75, 0.0, 0.5, 0.5, 1.25, 1.0, 0.333, 0.667, 0.667, 1.0, 0.875, 0.0, 0.333, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.5, 0.556, 0.0, 0.667, 0.667, 0.0, 1.0, 0.0, 0.0, 0.333, 1.0, 1.0, 0.333, 0.333, 1.0, 0.5, 0.667, 0.667, 1.0, 0.5, 0.5, 0.667, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.2, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.75, 0.25, 0.5, 1.0, 0.0, 0.333, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.667, 0.0, 0.333, 0.333, 0.667, 0.778, 0.0, 0.0, 1.0, 0.0, 0.5, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.5, 0.75, 0.25, 0.667, 0.667, 0.667, 0.667, 0.667, 0.333, 0.333, 0.333, 0.667, 0.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.667, 0.333, 0.0, 0.333, 0.333, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.667, 0.667, 0.667, 1.0, 0.25, 0.0, 0.333, 0.667, 0.0, 0.5, 0.25, 0.0, 0.333, 1.0, 0.0, 0.333, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.333, 0.25, 0.333, 1.0, 0.0, 0.333, 0.0, 1.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.25, 0.333, 0.667, 0.667, 0.0, 1.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.667, 1.0, 0.667, 0.667, 0.0, 0.0, 0.333, 1.0, 0.0, 0.333, 1.0, 0.0, 0.667, 0.0, 0.667, 0.333, 0.0, 1.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 1.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.5, 1.0, 0.667, 1.0, 0.0, 0.25, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.0, 0.0, 0.0, 0.667, 0.5, 0.0, 0.0, 0.0, 0.5, 0.667, 1.0, 0.667, 1.0, 0.667, 0.333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.667, 1.0, 0.667, 0.25, 0.5, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.25, 0.0, 0.5, 0.333, 0.0, 0.333, 0.333, 0.333, 0.667, 0.333, 0.5, 1.0, 0.667, 0.667, 1.333, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.333, 0.667, 0.333, 0.333, 0.0, 0.667, 0.6, 0.667, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.667, 0.333, 0.0, 0.667, 0.333, 1.0, 0.0, 0.25, 0.667, 0.0, 1.0, 0.0, 0.0, 0.333, 1.0, 1.0, 0.333, 1.0, 1.0, 0.333, 0.333, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.667, 0.5, 0.0, 0.5, 0.25, 1.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 4.0, 0.333, 1.0, 1.333, 0.667, 1.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.5, 0.5, 0.5, 1.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 1.0, 0.5, 0.0, 0.333, 1.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 1.0, 0.0, 0.0, 0.25, 0.5, 0.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.667, 0.667, 1.0, 1.0, 0.667, 0.0, 0.75, 0.0, 0.75, 0.667, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.75, 0.25, 0.667, 0.667, 0.333, 0.0, 1.0, 0.333, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.0, 1.0, 1.0, 0.6, 0.0, 0.0, 0.667, 0.667, 0.667, 0.0, 0.0, 0.375, 1.0, 0.0, 0.0, 1.0, 0.667, 0.667, 0.6, 0.6, 0.333, 0.5, 0.667, 0.667, 0.333, 0.333, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.333, 0.333, 1.0, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.333, 0.333, 0.75, 0.0, 0.0, 1.333, 0.0, 0.667, 0.333, 0.333, 0.5, 0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.0, 0.0, 0.667, 0.333, 0.5, 0.0, 0.25, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 0.25, 0.0, 0.25, 0.0, 0.333, 0.5, 0.0, 0.0, 1.333, 1.0, 1.0, 0.667, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.25, 0.333, 0.333, 1.5, 1.5, 1.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 1.0, 1.0, 0.333, 0.5, 0.0, 0.333, 0.333, 0.818, 0.364, 0.182, 1.0, 0.667, 0.667, 0.333, 0.667, 0.667, 1.0, 1.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 1.333, 1.333, 0.5, 0.667, 1.0, 1.333, 0.0, 0.0, 0.2, 0.2, 0.333, 0.0, 1.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 1.0, 0.0, 1.0, 0.0, 0.8, 0.0, 0.0, 0.25, 0.5, 0.0, 0.25, 0.0, 0.0, 1.0, 0.667, 0.0, 0.333, 0.333, 0.0, 0.667, 1.0, 0.333, 0.0, 0.0, 1.333, 0.75, 1.0, 0.0, 0.333, 0.333, 0.0, 0.5, 0.667, 0.0, 0.0, 0.667, 0.333, 0.667, 0.0, 0.333, 0.333, 1.0, 1.0, 0.0, 0.5, 1.0, 0.0, 0.667, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 1.5, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.75, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.333, 0.333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.333, 0.667, 1.0, 1.667, 1.0, 1.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.0, 0.25, 0.667, 0.0, 0.0, 1.0, 1.0, 0.333, 0.0, 1.0, 0.0, 0.333, 0.333, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 1.0, 0.667, 0.667, 0.333, 0.0, 1.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.0, 1.5, 0.0, 0.333, 0.333, 0.5, 0.0, 0.667, 1.0, 1.0, 0.667, 1.0, 1.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 1.333, 0.667, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 1.0, 0.333, 0.0, 1.0, 0.333, 0.0, 0.0, 1.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.667, 0.0, 0.667, 1.0, 0.333, 0.333, 0.0, 0.667, 0.667, 0.333, 1.333, 0.667, 0.667, 1.0, 0.0, 0.333, 0.333, 0.0, 0.333, 1.0, 0.0, 1.0, 0.0, 0.667, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.5, 0.5, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.0, 0.333, 0.333, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.0, 0.667, 0.0, 0.5, 0.0, 0.5, 0.75, 0.25, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.333, 0.667, 0.333, 0.667, 0.0, 0.667, 1.0, 0.667, 0.0, 0.667, 1.0, 1.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.0, 0.5, 0.0, 0.0, 0.333, 0.667, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.5, 0.667, 0.0, 0.0, 0.5, 0.5, 0.0, 0.333, 0.0, 0.5, 0.0, 1.0, 0.333, 1.0, 0.333, 0.667, 0.333, 0.667, 0.333, 0.333, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 0.0, 0.667, 1.0, 1.0, 0.0, 0.333, 0.333, 0.5, 0.333, 0.667, 1.0, 1.0, 0.5, 0.667, 0.667, 0.0, 0.667, 0.667, 3.667, 0.333, 1.0, 1.333, 1.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.333, 0.333, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 1.0, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 2.0, 2.0, 0.333, 0.333, 0.5, 0.25, 0.75, 0.0, 1.0, 0.667, 0.667, 0.667, 0.667, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 1.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.75, 2.5, 0.75, 0.5, 0.0, 0.0, 0.333, 1.0, 0.0, 0.0, 0.333, 0.0, 2.333, 1.0, 0.333, 0.333, 0.333, 0.667, 0.0, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 1.0, 0.333, 0.333, 0.0, 0.667, 1.0, 0.667, 0.333, 0.182, 0.636, 0.273, 1.333, 1.0, 0.667, 0.667, 1.333, 0.333, 0.667, 0.667, 0.667, 0.333, 0.0, 0.5, 0.0, 0.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.667, 0.0, 0.0, 1.0, 0.333, 0.667, 1.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.0, 0.333, 0.667, 0.333, 1.0, 1.0, 0.5, 0.5, 0.0, 1.0, 1.0, 0.5, 0.75, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.333, 1.0, 0.667, 0.0, 1.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.75, 0.0, 0.0, 0.0, 0.0, 0.333, 1.0, 0.333, 0.667, 0.333, 0.0, 0.0, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.667, 0.5, 0.5, 0.5, 1.667, 0.667, 0.667, 0.667, 0.0, 0.75, 1.0, 0.333, 0.667, 0.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.5, 0.0, 0.667, 0.667, 0.0, 0.0, 0.333, 0.667, 0.333, 1.0, 0.333, 0.667, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.5, 0.333, 0.333, 0.667, 1.0, 0.0, 1.0, 0.333, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.75, 1.0, 0.75, 0.667, 0.0, 0.0, 0.0, 0.0, 0.667, 1.0, 1.5, 0.5, 1.5, 0.333, 0.0, 0.333, 0.667, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.667, 0.5, 0.667, 0.667, 0.667, 0.0, 0.5, 0.5, 0.75, 0.25, 0.25, 1.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.333, 1.0, 0.75, 0.667, 0.0, 1.0, 0.0, 0.5, 0.333, 2.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.667, 0.733, 0.333, 0.333, 0.333, 0.667, 0.5, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.667, 0.667, 1.0, 0.667, 0.0, 0.333, 0.0, 0.667, 0.333, 0.667, 0.0, 0.333, 0.667, 0.0, 1.0, 0.333, 1.0, 0.667, 0.333, 0.667, 0.5, 0.5, 0.667, 0.333, 1.5, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.25, 0.333, 0.667, 0.0, 0.25, 0.0, 0.667, 0.0, 0.333, 0.0, 0.333, 0.333, 1.0, 0.5, 0.667, 0.0, 1.0, 1.0, 1.5, 0.5, 1.0, 0.5, 0.5, 0.0, 2.0, 0.5, 0.333, 0.667, 0.0, 0.333, 0.333, 0.333, 0.333, 1.667, 0.333, 1.0, 1.0, 0.333, 0.5, 0.5, 0.667, 0.333, 1.0, 0.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.667, 0.5, 0.5, 0.0, 1.5, 0.5, 0.5, 0.5, 0.333, 0.333, 0.667, 1.5, 1.5, 1.0, 0.333, 0.667, 0.667, 1.0, 0.667, 0.333, 0.667, 0.333, 0.667, 0.667, 1.0, 0.667, 0.333, 0.667, 0.667, 0.667, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.5, 0.333, 0.0, 0.0, 0.333, 0.667, 0.333, 0.333, 0.5, 0.5, 0.0, 3.0, 0.333, 0.0, 0.75, 0.667, 1.0, 1.0, 2.0, 0.667, 0.333, 0.667, 1.0, 0.0, 1.0, 1.0, 1.0, 0.333, 0.0, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.333, 0.2, 0.667, 0.667, 0.667, 0.667, 0.0, 0.5, 0.0, 1.0, 1.333, 0.0, 0.667, 0.667, 0.667, 0.667, 0.667, 0.667, 1.0, 0.0, 0.5, 0.5, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.273, 0.364, 0.333, 0.333, 0.333, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.5, 0.667, 0.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.333, 1.0, 0.667, 0.333, 0.5, 0.5, 0.5, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.75, 1.0, 0.0, 0.333, 0.667, 0.0, 0.5, 0.0, 0.0, 0.5, 0.333, 0.0, 0.0, 0.0, 0.5, 0.5, 0.333, 0.0, 0.5, 1.0, 0.5, 0.5, 0.333, 0.333, 0.0, 0.333, 0.667, 0.667, 0.333, 0.333, 1.0, 0.333, 1.0, 1.333, 0.667, 0.667, 0.667, 1.0, 0.333, 0.5, 1.0, 0.667, 0.333, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 1.0, 0.5, 1.0, 0.0, 0.0, 0.5, 0.5, 0.333, 0.667, 0.0, 0.333, 0.0, 0.5, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.667, 0.667, 1.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.667, 0.0, 0.0, 0.667, 0.667, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.667, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.667, 0.0, 0.0, 0.667, 1.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.5, 0.667, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.667, 1.0, 0.667, 1.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.667, 1.0, 0.333, 0.333, 0.667, 0.333, 0.5, 0.5, 0.5, 0.0, 0.667, 0.75, 0.5, 0.667, 0.333, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.667, 0.0, 0.333, 0.333, 0.333, 0.667, 0.667, 0.5, 0.0, 0.5, 0.333, 0.0, 1.0, 0.0, 1.0, 0.5, 0.333, 0.0, 0.667, 0.667, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.667, 1.0, 0.333, 1.0, 0.333, 0.667, 0.556, 0.0, 1.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.667, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.5, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 1.25, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.75, 0.583, 0.75, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 1.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.333, 0.333, 0.0, 0.333, 1.0, 0.0, 1.0, 1.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.667, 0.333, 0.667, 0.0, 0.25, 0.5, 0.75, 1.0, 1.333, 0.333, 0.333, 0.333, 0.333, 1.333, 0.333, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.333, 0.667, 0.5, 0.5, 0.0, 0.667, 0.333, 0.667, 0.0, 0.0, 1.0, 0.333, 0.333, 0.667, 0.333, 0.667, 0.0, 0.5, 0.5, 1.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.333, 0.0, 0.333, 1.0, 1.0, 1.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.333, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.333, 0.667, 0.333, 0.667, 0.5, 0.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 1.333, 0.667, 0.0, 0.667, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.667, 0.333, 0.0, 0.667, 1.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.786, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.333, 0.5, 1.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 1.0, 0.333, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.667, 0.0, 0.667, 1.0, 0.0, 0.5, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 1.333, 0.333, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.5, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 1.0, 0.667, 1.0, 0.667, 0.667, 1.5, 0.5, 0.667, 0.667, 1.0, 1.0, 0.333, 0.6, 0.8, 0.8, 0.667, 0.5, 1.0, 0.0, 0.667, 0.5, 0.5, 0.0, 0.5, 0.75, 0.333, 1.333, 1.333, 0.0, 0.5, 0.75, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.25, 0.333, 0.0, 0.0, 0.333, 0.7, 1.0, 0.25, 0.333, 1.0, 0.5, 1.0, 1.0, 1.0, 0.667, 1.0, 1.0, 0.75, 0.333, 1.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.667, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.0, 1.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.25, 0.5, 1.5, 0.5, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 1.0, 0.667, 0.667, 1.333, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, 0.0, 0.333, 0.667, 1.0, 0.667, 2.0, 1.0, 1.333, 0.0, 0.0, 0.333, 0.0, 0.5, 0.0, 0.333, 0.0, 1.0, 1.0, 0.5, 0.0, 0.769, 0.333, 0.667, 0.333, 0.75, 1.0, 0.333, 0.333, 0.333, 0.333, 0.667, 1.0, 1.0, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 1.333, 0.0, 1.5, 0.0, 0.0, 1.0, 0.0, 0.5, 0.667, 0.667, 0.364, 0.333, 0.333, 0.333, 0.667, 0.0, 0.0, 0.5, 1.0, 2.0, 0.0, 0.0, 0.0, 0.5, 0.75, 0.333, 0.0, 1.333, 1.5, 1.0, 1.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.333, 0.333, 0.667, 0.333, 1.0, 0.5, 0.333, 0.667, 0.667, 0.0, 0.667, 0.429, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.0, 0.667, 0.667, 0.333, 0.0, 1.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.667, 1.333, 0.0, 1.0, 0.0, 1.0, 0.0, 0.667, 0.333, 0.5, 0.0, 0.0, 0.0, 0.333, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 1.333, 0.667, 0.5, 0.5, 0.667, 1.333, 0.5, 1.5, 0.667, 0.0, 1.0, 1.0, 0.333, 0.5, 0.333, 0.0, 0.667, 1.0, 0.667, 0.667, 0.0, 1.0, 0.667, 0.5, 0.333, 0.333, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.75, 0.5, 0.667, 0.333, 0.333, 1.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.667, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.667, 0.5, 1.0, 0.5, 0.667, 0.0, 0.0, 1.0, 1.0, 0.667, 0.333, 1.0, 0.333, 0.0, 0.5, 0.333, 0.667, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 1.0, 0.0, 0.0, 1.333, 1.333, 0.667, 1.0, 0.667, 0.667, 0.667, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.444, 0.0, 0.667, 0.667, 0.333, 0.667, 0.667, 1.0, 0.75, 0.0, 0.5, 1.0, 0.5, 0.5, 0.0, 0.0, 1.0, 0.333, 1.0, 0.333, 1.0, 0.0, 0.667, 0.0, 0.333, 0.333, 1.0, 0.0, 0.0, 0.333, 1.0, 0.667, 0.0, 0.333, 1.5, 0.5, 2.333, 0.333, 0.333, 0.667, 1.0, 0.5, 1.0, 0.667, 0.0, 0.5, 1.333, 0.667, 0.667, 1.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.75, 0.0, 0.75, 0.333, 0.0, 0.0, 0.25, 1.0, 0.0, 0.5, 0.333, 0.778, 0.556, 0.778, 0.667, 0.6, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 1.0, 0.333, 0.667, 0.667, 0.333, 0.0, 0.571, 1.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.6, 0.2, 0.25, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 0.667, 0.333, 0.667, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.25, 0.0, 1.0, 1.333, 1.0, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.7, 0.9, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.667, 0.0, 0.667, 0.333, 1.0, 0.667, 0.667, 0.667, 0.333, 0.0, 1.0, 0.5, 0.5, 0.0, 1.0, 0.667, 0.0, 0.333, 0.667, 0.667, 0.667, 1.0, 0.0, 0.667, 0.333, 0.667, 0.5, 0.667, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.25, 1.25, 0.0, 0.667, 0.667, 0.333, 1.0, 0.0, 0.0, 0.667, 0.333, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.667, 0.333, 1.0, 1.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 1.0, 0.333, 0.667, 0.333, 0.0, 1.333, 1.333, 0.0, 0.0, 0.333, 0.333, 0.5, 0.333, 0.667, 0.333, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 1.5, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.75, 0.75, 0.5, 0.25, 0.75, 1.0, 0.75, 0.5, 0.333, 0.0, 0.0, 0.0, 0.0, 0.846, 0.923, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.333, 0.0, 0.0, 0.333, 0.667, 1.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.667, 0.0, 0.5, 1.0, 0.0, 0.5, 0.5, 0.333, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.667, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 1.0, 0.5, 0.5, 0.5, 0.667, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.125, 0.0, 0.0, 1.0, 0.25, 1.5, 0.0, 0.0, 1.0, 1.0, 0.667, 0.0, 1.0, 0.0, 0.333, 0.667, 1.0, 0.667, 0.667, 1.0, 1.0, 0.75, 0.5, 2.0, 1.5, 1.0, 1.0, 0.0, 1.0, 0.667, 0.667, 0.667, 0.333, 1.0, 0.0, 0.333, 0.5, 0.5, 0.0, 0.333, 0.333, 0.667, 0.0, 0.0, 0.667, 1.0, 0.667, 0.667, 0.0, 0.0, 1.0, 1.0, 0.667, 0.333, 0.0, 0.333, 0.333, 0.667, 0.111, 0.111, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.667, 1.0, 0.667, 0.5, 0.5, 0.5, 0.6, 0.667, 0.333, 0.333, 0.333, 1.0, 1.0, 0.0, 0.0, 0.5, 0.25, 0.667, 0.667, 0.333, 0.0, 0.333, 0.667, 0.667, 0.0, 1.0, 0.0, 0.333, 1.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 1.0, 1.333, 0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.333, 1.0, 0.0, 0.667, 0.667, 0.333, 0.667, 0.333, 1.333, 1.0, 0.333, 0.0, 0.5, 0.667, 1.333, 0.333, 0.333, 0.0, 0.667, 1.0, 1.667, 0.667, 0.333, 0.333, 0.333, 1.0, 0.667, 1.333, 1.0, 0.667, 1.0, 1.333, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.0, 0.5, 0.667, 0.667, 0.333, 1.0, 0.0, 0.667, 1.0, 0.333, 0.667, 0.667, 1.0, 1.0, 0.667, 0.25, 1.0, 0.333, 0.0, 1.0, 0.667, 0.667, 0.333, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.5, 0.5, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0, 0.5, 0.667, 0.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.333, 0.333, 1.0, 0.667, 0.333, 0.5, 0.0, 1.25, 1.0, 1.0, 0.333, 0.667, 0.333, 0.667, 1.0, 0.333, 1.0, 0.333, 0.667, 1.0, 1.0, 0.0, 0.667, 1.0, 0.667, 0.333, 0.333, 0.0, 0.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.333, 1.0, 0.333, 0.333, 0.333, 0.667, 0.333, 0.333, 0.667, 0.333, 1.0, 0.667, 0.333, 0.333, 0.333, 0.667, 1.0, 1.0, 1.0, 1.0, 0.333, 1.0, 0.0, 0.667, 0.667, 1.0, 0.333, 0.667, 0.667, 0.0, 1.333, 0.5, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.667, 0.5, 0.75, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.667, 1.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.333, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.333, 1.0, 0.667, 0.5, 0.5, 0.0, 0.5, 0.0, 0.667, 0.333, 0.667, 1.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 0.667, 0.333, 0.333, 0.0, 0.5, 0.5, 2.0, 1.5, 0.333, 0.333, 0.667, 0.667, 0.0, 0.333, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.667, 0.0, 0.667, 0.667, 0.667, 0.75, 0.25, 0.667, 0.0, 0.0, 0.5, 1.0, 0.667, 0.333, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.75, 0.25, 1.0, 1.0, 1.333, 1.0, 1.0, 0.667, 0.667, 0.0, 0.0, 1.0, 0.0, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.6, 0.0, 0.0, 0.0, 1.0, 1.0, 0.5, 0.333, 0.667, 0.333, 1.0, 1.5, 0.333, 0.0, 0.5, 0.0, 0.333, 0.333, 0.5, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.75, 0.667, 0.0, 1.0, 0.667, 0.333, 0.667, 0.333, 1.0, 1.0, 0.5, 1.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 1.0, 0.0, 0.0, 0.667, 0.667, 0.667, 1.333, 0.0, 0.333, 0.333, 1.0, 0.0, 0.333, 1.0, 1.0, 0.333, 0.5, 0.75, 0.0, 0.0, 1.0, 0.333, 1.0, 1.0, 0.333, 0.0, 1.0, 0.5, 0.667, 1.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.5, 0.0, 0.667, 0.333, 0.333, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 1.0, 1.0, 0.5, 0.5, 0.5, 0.0, 1.333, 0.333, 0.667, 0.0, 0.667, 0.667, 1.0, 0.0, 0.333, 0.667, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.5, 0.333, 0.0, 1.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.5, 0.5, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 1.0, 0.667, 0.727, 0.727, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 1.0, 0.667, 0.0, 0.667, 0.333, 0.5, 0.25, 0.333, 0.0, 0.0, 0.667, 0.333, 0.667, 0.333, 0.667, 1.5, 1.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.667, 0.667, 0.667, 0.0, 0.333, 0.0, 0.667, 1.0, 0.0, 1.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.667, 0.333, 0.25, 0.0, 1.0, 0.333, 0.0, 0.0, 0.667, 1.0, 0.667, 0.636, 0.0, 0.667, 2.0, 1.0, 0.0, 1.0, 1.333, 0.667, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.333, 0.333, 0.0, 0.0, 1.5, 0.0, 1.0, 1.0, 1.5, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.333, 0.333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.667, 0.333, 0.667, 0.333, 0.333, 1.0, 0.5, 0.5, 1.0, 0.333, 0.333, 0.333, 0.667, 0.333, 0.0, 0.333, 0.75, 0.667, 0.333, 0.5, 0.667, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.333, 0.333, 0.0, 0.0, 1.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.667, 0.0, 0.333, 0.333, 1.0, 0.0, 1.0, 1.0, 0.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.75, 0.0, 0.0, 0.0, 0.333, 1.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.75, 0.333, 0.333, 0.667, 0.667, 0.0, 0.667, 0.667, 0.0, 0.333, 0.667, 0.667, 0.333, 0.333, 0.75, 0.333, 0.333, 0.0, 0.0, 0.5, 0.333, 0.667, 1.333, 0.75, 0.667, 1.5, 0.0, 0.667, 0.667, 0.0, 0.667, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.5, 0.0, 0.0, 0.667, 0.0, 0.333, 0.667, 0.333, 1.0, 0.5, 0.0, 0.25, 0.333, 0.333, 0.0, 0.0, 1.0, 0.333, 0.333, 0.333, 0.667, 0.0, 0.333, 0.0, 0.0, 0.333, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.667, 1.0, 0.0, 0.333, 1.0, 0.333, 0.0, 0.667, 1.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.2, 0.0, 1.333, 0.333, 0.0, 0.0, 2.0, 0.25, 0.667, 0.667, 0.0, 1.0, 0.5, 0.25, 0.5, 0.5, 1.0, 1.0, 0.7, 0.25, 0.5, 0.5, 0.667, 1.0, 0.0, 0.2, 0.8, 0.2, 0.25, 0.5, 0.333, 0.667, 0.333, 1.333, 0.333, 0.333, 0.667, 0.0, 0.667, 0.333, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 1.0, 0.333, 0.667, 1.0, 1.0, 0.667, 0.667, 0.333, 0.667, 1.0, 0.5, 0.0, 0.667, 0.333, 0.0, 1.0, 0.0, 5.0, 0.0, 0.0, 0.333, 0.0, 0.25, 0.25, 0.75, 0.0, 0.25, 0.25, 0.5, 1.667, 0.0, 0.667, 0.667, 0.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.333, 0.5, 1.5, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.5, 1.0, 0.0, 0.667, 1.5, 1.0, 1.5, 0.333, 0.667, 1.667, 1.0, 0.25, 0.75, 0.75, 1.5, 0.5, 0.667, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.5, 0.667, 0.333, 0.0, 0.0, 0.5, 0.0, 0.667, 0.25, 0.333, 0.667, 0.0, 0.0, 0.333, 0.667, 1.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 1.0, 0.667, 0.667, 0.25, 1.0, 0.0, 0.667, 0.0, 0.75, 0.0, 0.333, 1.0, 2.0, 1.0, 0.0, 0.667, 0.667, 0.0, 1.0, 0.667, 0.667, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.667, 0.0, 0.333, 0.222, 2.0, 1.0, 0.667, 0.333, 0.0, 1.0, 0.333, 0.5, 0.0, 1.0, 0.5, 0.75, 0.667, 1.0, 0.333, 1.0, 1.0, 0.667, 1.0, 0.333, 0.333, 0.333, 0.0, 0.667, 0.333, 0.5, 0.0, 0.5, 0.429, 0.5, 1.0, 1.5, 1.0, 1.0, 0.667, 0.667, 0.0, 0.5, 0.0, 1.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.25, 0.25, 2.25, 1.0, 0.333, 1.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 1.0, 1.333, 0.333, 0.0, 0.667, 1.333, 0.5, 0.0, 0.333, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.333, 1.0, 0.0, 0.333, 0.333, 0.5, 0.5, 0.0, 0.333, 0.0, 0.5, 1.5, 0.0, 1.5, 0.25, 0.0, 0.5, 0.333, 0.571, 1.143, 0.857, 0.714, 0.857, 0.0, 0.667, 0.0, 0.5, 1.0, 0.5, 1.5, 0.0, 0.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.667, 1.333, 0.333, 0.0, 0.333, 1.0, 1.0, 1.333, 1.0, 1.0, 1.0, 1.0, 1.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.667, 0.333, 0.0, 0.5, 0.0, 1.0, 2.0, 1.0, 0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.0, 1.0, 0.5, 0.25, 0.333, 0.333, 0.667, 0.0, 0.333, 1.0, 0.333, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 1.333, 1.333, 1.0, 1.0, 0.0, 0.333, 0.667, 1.0, 0.667, 0.0, 0.0, 0.5, 0.0, 0.333, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 1.0, 0.0, 0.333, 0.667, 0.333, 0.0, 1.0, 1.333, 0.667, 1.0, 0.667, 0.0, 0.0, 1.333, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.5, 0.5, 0.0, 1.0, 1.5, 0.333, 0.333, 0.333, 0.0, 0.667, 1.0, 0.333, 1.0, 0.333, 0.667, 0.667, 0.667, 0.0, 1.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.0, 0.667, 0.333, 0.333, 1.5, 1.0, 0.333, 0.333, 0.0, 0.667, 0.667, 0.333, 0.333, 1.0, 0.0, 0.333, 0.333, 0.667, 0.0, 0.667, 0.667, 0.667, 0.75, 1.0, 0.667, 0.667, 0.333, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.25, 0.75, 0.333, 0.333, 0.667, 0.889, 0.0, 0.667, 0.0, 0.75, 0.75, 0.5, 0.25, 0.333, 0.333, 0.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.667, 0.333, 0.714, 0.857, 0.75, 0.667, 0.667, 0.667, 0.333, 1.0, 1.0, 0.667, 0.667, 0.333, 0.667, 0.0, 0.333, 0.333, 0.0, 0.5, 0.75, 0.0, 1.0, 0.5, 0.333, 0.667, 1.0, 0.667, 0.667, 0.0, 1.0, 0.0, 0.667, 0.667, 0.333, 1.0, 0.333, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.333, 0.0, 0.5, 0.5, 0.333, 0.333, 0.667, 0.0, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.333, 0.333, 0.5, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.5, 1.0, 1.0, 0.667, 1.0, 1.5, 1.5, 0.5, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.333, 1.0, 0.667, 0.333, 0.667, 0.667, 1.0, 1.0, 0.5, 1.0, 0.667, 0.667, 0.923, 0.667, 1.0, 0.333, 0.333, 1.0, 0.667, 0.667, 0.667, 1.0, 0.333, 1.333, 0.333, 0.0, 0.2, 0.2, 0.333, 0.333, 0.667, 0.0, 0.0, 0.5, 0.333, 1.0, 1.0, 0.667, 0.333, 0.5, 0.5, 0.5, 0.5, 0.75, 0.667, 1.0, 0.5, 1.333, 0.333, 0.333, 0.667, 0.5, 1.5, 0.667, 0.667, 0.667, 0.667, 0.667, 0.667, 1.333, 0.333, 0.667, 1.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.667, 0.0, 0.667, 1.0, 1.333, 0.667, 1.0, 0.667, 0.667, 1.0, 1.0, 0.333, 0.0, 0.5, 0.5, 0.5, 0.333, 0.0, 0.333, 1.0, 0.333, 1.333, 0.667, 0.333, 0.222, 1.0, 0.667, 0.667, 1.0, 0.333, 0.0, 0.0, 1.0, 0.333, 0.667, 0.333, 0.667, 0.667, 0.333, 0.667, 0.333, 1.0, 0.25, 0.0, 0.75, 0.0, 0.333, 0.75, 1.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.333, 1.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.25, 0.25, 0.333, 1.333, 0.333, 0.333, 0.667, 0.667, 0.0, 0.5, 1.0, 0.5, 0.667, 0.333, 1.0, 1.0, 0.636, 0.909, 0.333, 1.0, 1.0, 1.0, 0.333, 0.667, 0.5, 0.333, 1.333, 1.0, 0.667, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.25, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.75, 0.25, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 0.0, 0.0, 0.0, 1.5, 0.333, 0.667, 0.667, 0.667, 1.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.333, 1.0, 0.0, 0.667, 0.333, 1.333, 0.667, 0.0, 0.0, 0.0, 0.333, 1.333, 0.0, 0.0, 0.333, 0.667, 0.0, 0.667, 0.333, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.333, 0.0, 1.0, 0.667, 0.667, 0.667, 0.667, 0.667, 0.667, 0.333, 0.5, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.286, 1.0, 0.0, 0.333, 0.667, 0.667, 1.0, 1.0, 0.667, 0.333, 0.5, 0.25, 0.25, 0.0, 0.5, 0.5, 0.333, 0.0, 1.0, 1.0, 0.667, 0.667, 1.0, 0.667, 0.0, 0.333, 0.667, 1.0, 1.0, 1.333, 0.333, 0.333, 0.0, 0.667, 0.333, 0.667, 0.333, 1.0, 0.5, 0.0, 0.0, 0.0, 0.636, 0.667, 0.5, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.25, 0.5, 0.5, 0.0, 0.0, 0.5, 1.0, 0.5, 0.0, 1.0, 1.0, 2.0, 0.0, 0.333, 0.5, 0.75, 0.0, 0.333, 1.0, 1.0, 0.5, 0.667, 0.333, 0.333, 0.667, 0.0, 0.667, 0.5, 0.75, 0.5, 0.333, 0.0, 0.25, 0.75, 0.5, 0.667, 0.5, 1.5, 0.5, 0.5, 1.0, 0.25, 0.0, 1.0, 1.0, 0.333, 0.5, 1.0, 0.5, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.333, 1.0, 1.0, 0.667, 0.333, 0.667, 0.333, 0.333, 0.333, 1.5, 1.0, 1.0, 1.5, 0.333, 0.667, 0.0, 0.5, 0.0, 0.667, 0.5, 0.0, 0.0, 1.0, 1.0, 0.333, 0.333, 0.333, 0.667, 1.0, 0.667, 1.333, 1.0, 0.333, 1.0, 0.5, 0.333, 1.333, 0.333, 1.222, 0.0, 0.667, 0.667, 0.667, 3.333, 1.5, 0.667, 1.0, 0.333, 1.0, 1.0, 0.75, 0.25, 0.25, 0.5, 0.25, 1.333, 0.333, 0.333, 0.667, 1.0, 0.667, 0.333, 0.333, 0.333, 0.5, 0.333, 1.0, 0.0, 1.0, 0.667, 0.667, 0.667, 0.5, 0.0, 0.0, 0.333, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.5, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.25, 0.0, 0.0, 1.0, 0.5, 0.0, 0.667, 1.0, 1.333, 1.0, 0.5, 0.5, 0.333, 0.667, 1.0, 0.5, 0.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.5, 0.333, 0.0, 1.0, 1.5, 0.5, 0.0, 1.5, 1.5, 0.75, 0.5, 1.0, 0.0, 0.667, 0.5, 1.0, 1.0, 0.333, 0.0, 0.333, 0.333, 0.667, 0.667, 0.0, 0.333, 1.0, 1.0, 1.0, 1.0, 1.333, 1.0, 0.667, 0.5, 1.0, 0.667, 1.0, 1.0, 0.0, 1.0, 1.0, 1.5, 1.667, 0.667, 0.5, 0.5, 0.0, 1.0, 0.636, 0.333, 0.5, 0.667, 0.0, 0.0, 0.5, 0.0, 0.5, 0.333, 1.0, 1.0, 1.5, 0.5, 1.0, 0.0, 0.0, 1.0, 1.0, 0.5, 0.667, 1.0, 0.333, 1.5, 1.5, 0.5, 1.5, 0.5, 0.667, 0.0, 0.667, 1.0, 0.333, 0.333, 1.0, 1.0, 1.0, 0.333, 1.333, 0.667, 0.333, 1.0, 0.0, 0.333, 1.5, 1.5, 1.5, 1.0, 0.667, 0.5, 0.0, 0.667, 1.0, 1.0, 0.333, 0.0, 0.25, 0.333, 0.0, 0.667, 1.0, 1.0, 0.5, 0.5, 0.333, 0.333, 0.333, 0.333, 1.5, 0.0, 0.0, 0.0, 0.5, 0.667, 0.0, 0.333, 1.0, 0.667, 1.0, 0.333, 0.333, 0.5, 1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 0.333, 0.333, 0.5, 1.0, 0.5, 1.0, 0.0, 1.0, 0.0, 0.0, 1.667, 0.667, 0.0, 0.0, 0.333, 0.667, 1.0, 1.0, 1.333, 0.333, 0.9, 1.4, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 0.667, 1.333, 1.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.667, 1.0, 1.5, 1.0, 1.0, 0.333, 0.333, 1.0, 0.667, 0.667, 0.0, 0.0, 0.667, 1.0, 0.667, 0.0, 0.0, 0.667, 1.0, 1.0, 1.0, 0.5, 1.5, 0.667, 0.333, 1.0, 0.667, 0.667, 0.333, 0.333, 0.667, 1.0, 0.0, 0.667, 0.333, 1.0, 0.333, 0.667, 1.0, 1.333, 1.5, 1.0, 0.667, 0.333, 0.333, 0.333, 1.0, 0.667, 0.0, 1.333, 1.0, 0.667, 0.333, 0.667, 1.0, 1.0, 0.5, 0.333, 0.333, 1.0, 1.0, 1.0, 1.0, 0.25, 0.667, 0.333, 1.0, 0.333, 0.0, 0.667, 0.667, 0.0, 0.5, 0.0, 0.5, 0.333, 0.667, 0.333, 0.333, 1.5, 0.0, 1.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.0, 0.0, 1.5, 1.0, 0.0, 1.5, 0.667, 0.667, 0.333, 0.0, 1.0, 0.0, 1.0, 1.5, 0.75, 0.333, 0.0, 1.0, 0.667, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.333, 1.0, 0.333, 0.333, 0.0, 0.667, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 0.25, 0.5, 1.0, 1.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.667, 0.667, 0.667, 0.5, 0.0, 0.0, 1.0, 1.0, 0.667, 0.0, 0.667, 0.333, 0.667, 1.5, 1.5, 0.333, 0.333, 0.333, 1.5, 0.0, 0.667, 0.0, 0.667, 0.0, 0.333, 0.333, 0.333, 0.333, 0.333, 1.0, 1.0, 1.333, 1.0, 0.5, 0.0, 0.333, 0.333, 0.333, 0.333, 1.0, 0.0, 2.0, 0.5, 0.667, 0.0, 1.333, 0.667, 0.667, 0.333, 0.333, 1.0, 0.5, 0.5, 1.5, 0.5, 1.5, 0.75, 0.333, 0.5, 0.0, 0.333, 0.333, 0.0, 0.333, 1.0, 0.667, 0.5, 0.5, 1.0, 0.333, 1.0, 1.0, 0.333, 0.667, 0.333, 0.333, 0.333, 0.333, 0.333, 0.5, 1.0, 0.333, 0.333, 1.0, 0.7, 0.333, 0.333, 0.667, 2.0, 1.0, 0.25, 0.75, 0.667, 0.0, 0.5, 0.5, 1.333, 1.143, 0.5, 0.5, 0.5, 0.25, 0.333, 0.667, 1.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.667, 1.0, 0.75, 0.25, 1.0, 1.0, 0.667, 0.0, 0.667, 0.667, 1.0, 0.333, 0.667, 1.0, 0.333, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.5, 0.667, 1.333, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.667, 0.333, 0.333, 0.0, 0.333, 1.0, 1.0, 0.0, 0.667, 0.0, 1.0, 0.333, 1.333, 0.667, 0.5, 0.5, 0.8, 0.8, 1.6, 0.25, 0.0, 0.0, 1.0, 0.667, 1.0, 0.5, 0.333, 1.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.75, 0.25, 0.75, 0.25, 0.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.667, 1.0, 0.75, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.333, 0.333, 0.333, 0.333, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.5, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 1.0, 0.0, 0.5, 0.0, 0.75, 0.667, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.667, 0.667, 0.0, 0.333, 0.0, 1.5, 1.0, 0.0, 0.5, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.667, 1.0, 1.0, 0.667, 0.333, 0.333, 0.667, 0.667, 0.333, 0.333, 0.333, 0.667, 0.333, 0.333, 1.0, 1.333, 0.0, 0.5, 0.0, 1.0, 0.5, 1.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.5, 0.333, 0.667, 0.667, 0.333, 0.0, 1.0, 0.333, 1.0, 0.0, 1.0, 0.667, 0.333, 1.0, 0.5, 1.0, 1.0, 1.5, 0.667, 0.5, 0.5, 0.333, 1.0, 0.667, 0.667, 0.5, 0.333, 0.333, 0.0, 0.333, 0.333, 1.0, 0.667, 0.0, 0.0, 1.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.333, 0.333, 0.667, 0.333, 0.333, 0.333, 0.667, 0.333, 0.333, 0.0, 0.0, 0.538, 0.5, 0.5, 0.0, 0.667, 0.667, 1.0, 1.0, 1.5, 1.0, 0.667, 0.667, 0.333, 0.333, 0.667, 1.0, 1.0, 0.667, 0.0, 1.0, 1.333, 0.643, 0.667, 0.667, 0.667, 1.0, 0.0, 1.5, 1.0, 0.25, 0.444, 1.0, 1.0, 1.0, 0.333, 0.875, 1.0, 1.5, 0.556, 0.0, 0.5, 0.667, 1.333, 0.667, 0.667, 0.667, 0.0, 0.333, 0.667, 0.0, 0.667, 0.667, 1.0, 0.333, 1.0, 0.667, 0.25, 1.0, 1.0, 0.5, 0.0, 0.5, 0.333, 1.5, 0.333, 0.667, 1.0, 1.0, 0.0, 0.667, 0.0, 1.0, 0.333, 0.667, 1.0, 1.0, 0.333, 1.0, 0.667, 0.333, 0.667, 0.333, 0.333, 0.333, 0.333, 1.0, 0.333, 1.0, 0.0, 0.333, 0.667, 0.333, 1.333, 1.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.333, 1.0, 0.333, 0.0, 0.333, 0.0, 1.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.667, 0.0, 0.667, 0.333, 1.0, 0.5, 1.0, 0.667, 0.333, 0.667, 0.333, 1.333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.25, 0.0, 0.5, 0.333, 0.0, 0.0, 0.333, 1.0, 0.667, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.333, 1.0, 0.5, 0.5, 0.333, 0.0, 0.0, 1.0, 1.333, 1.333, 0.667, 0.0, 0.0, 0.667, 0.0, 0.333, 0.333, 1.0, 0.5, 0.0, 1.0, 0.667, 1.0, 1.0, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.667, 0.0, 0.333, 0.333, 0.5, 0.5, 0.0, 0.5, 0.25, 0.0, 0.667, 0.333, 0.0, 0.667, 1.0, 0.333, 0.75, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 1.0, 0.667, 1.0, 0.333, 1.0, 0.0, 0.5, 0.5, 0.333, 1.0, 1.0, 0.667, 0.667, 1.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.25, 1.0, 0.333, 0.333, 0.5, 0.5, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.0, 1.0, 0.0, 0.75, 1.0, 0.667, 0.667, 0.667, 0.333, 0.5, 1.0, 1.0, 1.0, 0.5, 0.25, 0.25, 0.5, 0.5, 0.667, 0.333, 1.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.0, 1.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.25, 0.0, 0.0, 0.667, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.333, 1.0, 0.6, 0.333, 0.0, 0.667, 1.0, 0.667, 0.667, 0.25, 0.25, 1.0, 0.667, 0.667, 0.333, 0.0, 0.667, 1.0, 1.0, 1.0, 1.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.5, 0.0, 0.667, 1.0, 0.333, 1.0, 0.5, 0.667, 1.0, 0.75, 0.25, 0.0, 0.25, 0.0, 0.25, 1.0, 0.333, 0.667, 0.333, 1.0, 2.667, 0.333, 0.667, 1.0, 0.0, 0.0, 1.333, 0.0, 1.0, 0.333, 0.0, 0.0, 0.5, 0.667, 0.0, 0.0, 0.5, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.5, 0.667, 0.333, 1.0, 1.0, 1.0, 1.857, 1.0, 1.0, 0.0, 1.0, 0.5, 0.0, 0.25, 1.0, 0.667, 1.0, 0.667, 0.333, 1.333, 0.5, 1.0, 1.0, 1.0, 1.0, 0.667, 0.667, 1.5, 1.0, 0.0, 0.5, 0.333, 0.667, 0.333, 0.667, 0.667, 0.667, 0.333, 0.333, 0.667, 0.667, 0.667, 1.0, 0.0, 0.667, 0.333, 0.333, 0.333, 1.0, 1.667, 0.0, 1.75, 1.25, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.5, 0.667, 1.0, 0.333, 0.333, 0.5, 0.333, 0.5, 0.5, 0.5, 1.0, 1.0, 1.5, 0.5, 1.5, 1.0, 1.0, 0.5, 0.0, 0.333, 0.0, 0.333, 0.333, 1.0, 0.333, 0.333, 0.75, 0.25, 1.0, 0.333, 1.0, 0.333, 0.0, 0.333, 1.333, 1.0, 0.667, 0.0, 0.333, 0.333, 0.667, 1.0, 0.667, 0.333, 0.333, 0.875, 0.667, 0.333, 0.667, 0.667, 0.667, 0.0, 0.667, 1.0, 0.0, 1.0, 0.667, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 1.0, 0.667, 0.667, 1.0, 0.25, 0.25, 0.0, 0.333, 0.0, 0.5, 0.75, 1.0, 0.667, 0.667, 1.0, 1.0, 1.0, 1.0, 0.667, 0.5, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.333, 1.0, 0.5, 1.0, 1.0, 0.5, 0.333, 0.667, 0.333, 0.333, 0.667, 0.333, 0.333, 0.667, 1.0, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 1.0, 0.333, 0.0, 1.0, 0.0, 0.333, 0.333, 1.5, 1.0, 0.333, 0.667, 0.667, 0.333, 0.333, 1.0, 0.667, 0.667, 0.333, 0.0, 1.0, 0.0, 0.333, 1.0, 1.333, 1.0, 0.667, 1.0, 1.0, 0.667, 0.0, 0.667, 1.333, 0.0, 0.0, 0.333, 0.667, 0.25, 1.0, 0.667, 0.333, 1.0, 0.667, 0.0, 1.333, 1.0, 0.667, 0.667, 0.667, 0.333, 0.667, 1.0, 1.0, 0.667, 0.0, 0.0, 0.0, 0.667, 1.0, 1.333, 1.333, 1.0, 0.333, 0.333, 0.333, 0.333, 1.0, 1.5, 0.5, 0.0, 1.0, 1.0, 1.333, 0.333, 0.667, 1.333, 1.0, 0.5, 1.0, 0.333, 1.0, 0.333, 0.667, 0.333, 0.333, 0.667, 0.667, 1.0, 0.333, 0.667, 0.5, 1.0, 0.0, 0.667, 0.667, 0.667, 0.667, 0.333, 0.0, 0.667, 1.0, 0.5, 0.667, 0.333, 0.333, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.583, 1.0, 0.667, 0.667, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.0, 0.667, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.923, 0.0, 0.0, 0.0, 0.0, 0.75, 0.25, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.667, 1.0, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.333, 1.0, 0.667, 0.333, 0.778, 0.8, 1.333, 0.0, 1.333, 0.75, 1.0, 0.75, 1.0, 1.0, 0.75, 0.25, 0.25, 0.667, 0.333, 0.0, 1.0, 0.333, 0.333, 0.333, 0.5, 0.0, 0.0, 1.0, 1.5, 1.0, 0.5, 0.333, 0.667, 0.667, 0.0, 1.0, 1.333, 0.667, 0.667, 1.5, 0.667, 0.667, 0.333, 0.667, 0.333, 1.0, 0.667, 1.0, 0.667, 1.333, 1.0, 0.667, 0.5, 0.5, 0.0, 0.667, 1.0, 0.333, 0.667, 0.333, 0.5, 0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.333, 0.333, 1.0, 0.5, 0.0, 1.111, 0.667, 0.667, 0.667, 1.333, 0.667, 0.667, 0.75, 1.0, 0.333, 0.0, 1.0, 0.667, 1.5, 0.0, 0.5, 0.5, 1.0, 1.0, 0.667, 0.667, 0.667, 0.667, 0.0, 0.333, 0.25, 0.25, 1.0, 0.0, 0.667, 0.333, 0.333, 0.333, 0.333, 0.667, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.333, 0.333, 0.667, 0.333, 0.75, 0.5, 0.0, 0.667, 0.0, 0.333, 1.0, 1.0, 1.0, 0.667, 0.667, 0.333, 1.0, 1.667, 0.667, 1.0, 0.667, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 0.333, 0.333, 0.0, 0.0, 0.333, 0.444, 1.0, 1.0, 1.0, 0.667, 0.0, 0.333, 1.333, 1.0, 0.667, 1.0, 1.5, 1.5, 1.0, 0.333, 1.0, 0.333, 1.0, 1.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.667, 1.0, 0.0, 0.333, 0.667, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.0, 1.0, 0.667, 0.333, 0.333, 0.667, 1.0, 1.0, 1.0, 0.333, 0.667, 0.333, 1.0, 0.667, 0.5, 1.0, 1.0, 1.0, 0.333, 0.0, 0.667, 0.667, 0.667, 1.0, 1.0, 0.667, 0.0, 0.667, 0.333, 0.667, 0.0, 0.667, 0.0, 0.667, 0.0, 1.0, 0.333, 0.333, 0.0, 0.333, 0.5, 0.0, 0.5, 0.5, 0.333, 0.667, 0.333, 0.333, 0.667, 1.0, 1.333, 1.0, 0.333, 0.333, 0.333, 1.0, 0.0, 0.0, 0.5, 0.75, 1.0, 0.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.333, 0.333, 0.333, 0.333, 0.25, 0.5, 0.667, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 1.0, 1.0, 0.667, 1.0, 0.0, 0.667, 0.667, 0.0, 0.667, 1.0, 1.0, 1.0, 1.0, 0.333, 0.333, 0.333, 0.667, 0.667, 0.333, 0.667, 1.0, 0.667, 1.0, 0.25, 0.333, 0.667, 0.667, 0.333, 0.0, 0.333, 0.0, 0.333, 0.667, 1.333, 1.0, 0.333, 0.0, 0.333, 0.333, 0.333, 1.0, 0.75, 1.0, 0.667, 0.333, 1.0, 0.333, 0.667, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.667, 0.333, 0.667, 0.667, 0.667, 0.333, 0.333, 1.0, 1.0, 0.75, 1.0, 1.0, 0.75, 1.0, 0.75, 1.5, 1.0, 1.0, 1.0, 0.333, 0.333, 0.667, 0.667, 1.0, 0.667, 0.0, 0.333, 0.333, 1.0, 0.5, 0.667, 0.333, 0.5, 0.667, 0.667, 0.667, 1.0, 0.0, 0.333, 0.333, 1.0, 1.0, 0.667, 0.333, 1.3, 0.333, 1.0, 0.5, 0.667, 0.667, 1.0, 0.333, 0.0, 1.0, 0.5, 1.0, 0.25, 0.5, 0.0, 0.333, 0.0, 0.333, 1.0, 0.333, 0.0, 0.667, 0.667, 1.0, 0.5, 0.0, 0.333, 1.333, 0.667, 0.0, 0.333, 0.667, 0.333, 0.333, 0.667, 0.667, 1.0, 1.0, 1.0, 0.667, 0.667, 0.5, 0.5, 1.0, 1.0, 0.5, 1.5, 0.5, 1.0, 0.667, 0.0, 0.5, 1.0, 0.5, 0.0, 0.0, 2.5, 0.0, 0.333, 0.333, 0.667, 0.333, 1.0, 1.0, 0.667, 0.0, 0.333, 0.0, 0.333, 1.5, 0.5, 1.333, 1.0, 1.0, 0.0, 0.667, 0.333, 0.727, 0.0, 0.333, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.667, 0.667, 1.0, 0.667, 1.5, 0.5, 1.0, 1.0, 1.0, 0.333, 0.667, 0.333, 1.0, 0.333, 1.0, 1.0, 0.333, 0.333, 1.0, 0.333, 0.0, 0.5, 0.667, 0.0, 0.667, 0.333, 0.333, 0.667, 0.667, 0.0, 1.0, 0.333, 0.333, 0.667, 0.333, 1.5, 1.5, 0.0, 0.333, 0.0, 0.0, 0.5, 0.0, 1.0, 0.667, 0.333, 1.0, 1.0, 0.333, 0.5, 0.5, 0.333, 0.333, 1.0, 0.667, 0.667, 0.333, 0.333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.333, 0.667, 1.333, 0.333, 1.333, 1.333, 1.0, 1.0, 0.333, 0.667, 0.625, 0.667, 1.0, 1.5, 1.0, 0.333, 0.667, 0.333, 0.667, 1.0, 1.333, 1.0, 1.333, 0.667, 0.333, 0.0, 0.667, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.667, 1.0, 0.667, 0.0, 0.667, 0.0, 1.0, 0.667, 0.0, 0.333, 1.0, 0.333, 0.667, 0.0, 0.0, 0.5, 0.333, 0.667, 0.333, 0.333, 0.333, 0.333, 0.333, 0.0, 0.667, 1.0, 1.0, 0.333, 0.667, 0.333, 1.5, 0.333, 1.0, 1.0, 0.0, 1.333, 1.0, 0.667, 1.0, 0.667, 0.5, 0.5, 0.5, 0.5, 0.0, 0.333, 0.333, 0.0, 0.0, 1.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.75, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 0.667, 0.5, 0.0, 1.0, 0.5, 0.5, 0.5, 0.25, 0.0, 0.333, 0.333, 1.0, 1.0, 0.0, 0.667, 0.667, 1.333, 0.333, 1.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.333, 0.333, 1.0, 1.0, 1.0, 0.0, 0.0, 0.667, 0.333, 0.667, 0.0, 0.667, 1.0, 0.667, 0.75, 0.5, 0.333, 1.667, 0.333, 0.5, 0.5, 0.875, 0.667, 0.333, 0.333, 0.0, 1.0, 0.5, 0.25, 0.5, 0.5, 1.0, 0.5, 0.5, 0.0, 0.0, 1.0, 0.333, 0.333, 0.667, 1.0, 1.5, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.667, 0.0, 0.333, 0.0, 0.5, 0.0, 1.0, 0.5, 0.333, 0.5, 0.25, 1.0, 0.0, 0.0, 1.5, 1.0, 1.0, 0.75, 0.75, 0.75, 0.0, 1.5, 0.0, 0.5, 0.0, 0.5, 0.667, 1.0, 1.0, 0.667, 0.667, 0.5, 0.5, 0.5, 0.333, 0.333, 0.333, 0.0, 0.333, 0.333, 0.667, 0.333, 1.0, 1.0, 0.615, 0.333, 0.333, 1.0, 0.0, 0.667, 4.667, 0.333, 0.333, 0.333, 1.182, 0.333, 0.333, 1.333, 0.5, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.0, 0.667, 0.667, 0.5, 0.0, 1.0, 1.5, 0.333, 1.0, 1.0, 1.5, 2.0, 0.25, 0.25, 1.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.667, 0.333, 0.333, 0.667, 0.0, 0.333, 0.0, 1.0, 1.0, 0.667, 0.333, 0.0, 0.5, 1.5, 1.5, 0.667, 1.0, 0.667, 1.0, 0.667, 1.0, 1.0, 1.0, 0.0, 1.667, 0.0, 1.0, 0.75, 0.0, 0.667, 0.667, 0.0, 0.75, 0.5, 0.0, 0.667, 0.667, 0.667, 1.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 1.5, 0.5, 0.333, 0.667, 0.667, 0.333, 0.333, 0.667, 0.333, 1.0, 1.0, 0.667, 0.333, 0.667, 0.667, 0.5, 0.333, 1.333, 0.0, 1.0, 1.5, 0.0, 0.5, 0.333, 0.75, 1.0, 1.667, 0.333, 0.333, 0.5, 0.5, 0.667, 0.667, 1.0, 1.0, 0.333, 1.0, 0.333, 0.333, 1.333, 1.0, 0.667, 0.667, 0.333, 0.667, 1.0, 1.0, 0.667, 1.0, 0.667, 1.0, 0.5, 1.0, 0.0, 0.333, 0.667, 0.0, 0.0, 1.667, 0.0, 0.5, 0.667, 0.667, 1.0, 0.0, 1.0, 0.0, 0.667, 1.0, 1.0, 0.667, 1.0, 0.0, 0.333, 0.25, 0.5, 0.0, 0.0, 0.0, 1.0, 1.0, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.667, 0.25, 1.0, 1.333, 1.0, 0.667, 0.0, 1.0, 0.667, 1.333, 1.5, 1.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.333, 0.0, 0.667, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.5, 0.667, 1.0, 0.75, 0.333, 0.333, 1.0, 1.5, 1.333, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.5, 0.667, 0.333, 1.333, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.333, 0.333, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.667, 0.667, 0.667, 1.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.333, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 1.0, 0.5, 0.5, 0.667, 0.667, 0.667, 0.0, 0.0, 0.75, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.333, 0.333, 0.333, 1.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.333, 0.333, 1.0, 0.667, 0.667, 0.667, 0.333, 0.333, 0.333, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 1.0, 0.333, 0.667, 0.0, 0.333, 0.333, 0.667, 0.333, 0.333, 0.0, 0.5, 0.0, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.333, 0.0, 0.333, 1.0, 0.0, 0.667, 0.333, 0.667, 0.0, 0.667, 0.333, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 1.0, 0.667, 0.333, 0.667, 1.0, 0.333, 0.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.25, 0.25, 0.25, 0.0, 0.75, 0.0, 0.5, 1.0, 1.0, 0.5, 1.0, 0.0, 0.0, 0.333, 0.667, 0.667, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.667, 1.0, 0.667, 0.0, 1.333, 0.25, 0.5, 0.75, 0.333, 0.0, 0.667, 0.667, 0.0, 0.0, 1.0, 0.667, 0.0, 0.0, 0.667, 2.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.5, 0.0, 1.0, 0.333, 0.667, 1.667, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 0.667, 0.5, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.667, 0.333, 0.667, 0.333, 0.667, 0.667, 0.333, 0.667, 0.667, 0.667, 0.0, 0.667, 0.667, 0.333, 0.0, 0.0, 1.333, 1.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 1.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 1.667, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.667, 0.333, 0.667, 0.667, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.333, 0.333, 1.0, 0.667, 0.0, 0.333, 0.0, 0.5, 0.5, 0.25, 0.0, 0.333, 0.667, 0.667, 0.5, 0.0, 1.0, 0.0, 0.5, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.667, 0.0, 1.0, 0.667, 0.0, 0.333, 1.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.5, 0.5, 0.75, 0.25, 0.5, 0.333, 0.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.0, 0.667, 0.667, 0.0, 0.667, 0.667, 1.0, 1.0, 2.0, 0.0, 1.333, 0.333, 0.0, 0.0, 0.833, 0.667, 0.333, 0.667, 1.0, 1.0, 0.667, 1.0, 0.667, 0.333, 0.0, 0.667, 0.333, 0.333, 0.0, 0.333, 0.333, 0.333, 1.0, 1.0, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.333, 0.333, 0.667, 0.0, 1.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.5, 0.25, 0.0, 0.5, 0.667, 0.333, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.333, 1.0, 0.333, 0.667, 0.333, 0.0, 0.667, 0.333, 1.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.333, 0.333, 0.0, 0.667, 0.0, 0.667, 1.0, 0.333, 0.667, 0.667, 0.667, 1.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.333, 0.667, 0.333, 0.333, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.333, 0.333, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.667, 0.667, 0.0, 0.667, 0.333, 1.333, 1.0, 0.333, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 0.0, 0.0, 0.333, 1.0, 0.333, 0.25, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.667, 0.667, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.5, 0.333, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.5, 0.333, 0.667, 0.0, 0.333, 0.333, 1.0, 1.333, 1.0, 0.667, 0.667, 0.667, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.333, 0.0, 0.667, 0.0, 0.667, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 1.333, 0.667, 0.0, 1.0, 0.333, 0.667, 1.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.667, 0.0, 0.0, 1.0, 0.667, 0.667, 0.0, 0.667, 0.0, 0.5, 0.0, 0.667, 0.5, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.667, 1.0, 1.0, 0.333, 0.333, 1.0, 0.0, 0.0, 0.0, 1.0, 0.667, 1.0, 0.667, 0.333, 0.0, 0.0, 1.0, 0.667, 0.333, 0.5, 0.333, 0.333, 0.333, 1.333, 1.5, 0.5, 0.333, 0.333, 0.667, 0.333, 0.667, 0.0, 0.333, 0.333, 1.0, 0.214, 1.0, 0.333, 0.667, 0.667, 0.667, 0.333, 0.333, 0.333, 1.0, 0.0, 0.0, 1.0, 0.0, 0.667, 0.333, 0.667, 0.333, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.333, 0.5, 1.0, 1.0, 1.0, 0.0, 0.667, 0.0, 1.0, 0.333, 0.0, 0.0, 0.333, 0.333, 0.667, 0.0, 0.667, 0.667, 0.333, 0.333, 0.667, 0.333, 0.667, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.5, 0.0, 0.667, 1.0, 0.667, 0.667, 0.0, 0.0, 0.5, 0.0, 1.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.333, 0.667, 1.0, 0.0, 0.0, 1.0, 0.667, 0.333, 0.667, 0.0, 0.333, 0.5, 1.0, 0.5, 0.0, 1.0, 0.333, 1.0, 0.5, 0.667, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.667, 0.667, 0.0, 0.571, 0.714, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 0.0, 0.333, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.75, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.429, 0.857, 0.286, 0.286, 0.0, 0.0, 1.0, 0.667, 0.333, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.333, 0.667, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.667, 1.0, 0.333, 0.333, 0.333, 0.333, 0.0, 1.0, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 1.0, 0.333, 0.556, 0.0, 0.0, 0.0, 0.0, 0.75, 0.5, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.667, 0.667, 0.333, 0.667, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.667, 0.333, 0.333, 0.333, 0.333, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.667, 0.667, 0.667, 0.667, 0.667, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 1.333, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.333, 0.0, 0.333, 0.333, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 1.0, 0.667, 0.0, 0.667, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.5, 0.0, 0.333, 0.333, 0.667, 0.667, 0.333, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.333, 0.333, 0.0, 0.0, 0.667, 1.0, 0.0, 0.333, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 1.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.333, 0.0, 0.667, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.667, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.667, 0.333, 0.333, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 2.667, 0.0, 0.333, 0.0, 0.333, 0.333, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.667, 0.25, 0.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 1.0, 0.0, 0.667, 0.0, 0.333, 0.0, 0.667, 0.333, 1.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.0, 0.5, 0.0, 0.0, 0.0, 1.0, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.667, 1.0, 0.667, 0.333, 0.333, 0.0, 0.333, 0.333, 0.75, 0.5, 0.667, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 1.0, 0.0, 0.333, 0.0, 1.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 1.0, 1.2, 0.6, 0.667, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 1.2, 0.667, 1.444, 1.0, 1.5, 0.333, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 0.667, 0.333, 0.333, 0.667, 0.333, 1.0, 0.333, 0.667, 1.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.667, 1.0, 0.333, 0.0, 0.667, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.667, 0.333, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.333, 0.333, 0.667, 0.333, 1.0, 0.0, 0.333, 1.0, 0.333, 0.333, 0.25, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.667, 0.0, 0.0, 0.667, 0.333, 0.0, 0.0, 1.5, 0.5, 0.333, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.333, 0.5, 1.5, 1.0, 0.0, 0.0, 0.333, 1.5, 0.0, 0.0, 0.333, 0.667, 0.333, 1.0, 0.667, 0.0, 0.667, 0.667, 1.0, 0.5, 1.0, 0.8, 1.0, 0.333, 0.7, 0.0, 0.0, 0.0, 0.667, 0.0, 0.9, 1.0, 1.0, 1.0, 0.75, 0.333, 0.333, 0.667, 0.667, 0.5, 1.0, 0.5, 0.0, 0.333, 1.0, 0.667, 0.333, 1.0, 0.0, 0.2, 0.0, 0.667, 0.333, 0.667, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.333, 1.0, 0.0, 0.333, 0.0, 0.333, 0.667, 0.667, 0.333, 1.0, 0.667, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 1.0, 0.0, 0.75, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.75, 0.25, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.667, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 1.0, 0.667, 0.333, 0.0, 0.0, 0.0, 0.5, 0.0, 0.667, 0.25, 1.333, 0.333, 0.0, 0.0, 0.667, 0.667, 0.333, 0.333, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.6, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.091, 0.909, 0.667, 0.667, 1.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.333, 0.0, 0.0, 0.667, 0.0, 0.0, 0.733, 1.067, 0.333, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.667, 0.0, 0.846, 0.923, 0.5, 0.5, 0.583, 0.667, 0.667, 0.667, 0.333, 0.0, 0.143, 0.429, 0.286, 0.429, 0.0, 0.0, 0.667, 0.333, 0.0, 0.25, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.5, 0.5, 0.75, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.667, 0.0, 0.0, 0.667, 1.0, 1.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.667, 1.0, 1.0, 0.667, 0.667, 0.0, 0.917, 0.75, 0.583, 1.0, 1.0, 1.0, 0.0, 0.0, 0.333, 0.333, 0.667, 0.333, 0.333, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.0, 1.667, 0.667, 0.333, 0.667, 1.0, 0.0, 0.667, 0.667, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.0, 1.333, 0.333, 1.333, 1.0, 0.5, 0.5, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.667, 0.0, 0.333, 1.0, 0.667, 0.333, 1.0, 1.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 1.0, 1.333, 1.0, 0.0, 1.0, 0.0, 0.5, 0.667, 1.0, 0.667, 0.333, 1.0, 1.0, 0.667, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.667, 0.667, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.0, 0.333, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 1.0, 0.5, 0.0, 0.333, 0.333, 1.0, 1.667, 0.333, 0.0, 1.0, 1.667, 0.25, 0.5, 0.0, 0.667, 0.0, 0.0, 0.0, 0.111, 0.333, 0.333, 0.667, 0.0, 0.333, 0.75, 1.0, 0.667, 0.0, 0.667, 0.333, 0.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.333, 0.333, 0.0, 0.333, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.667, 1.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.333, 1.077, 0.0, 1.0, 0.667, 0.0, 0.0, 0.667, 0.0, 0.75, 0.0, 1.0, 0.0, 0.0, 0.333, 0.0, 0.0, 1.0, 0.667, 0.333, 0.308, 1.0, 0.6, 0.538, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.333, 0.0, 0.333, 0.5, 0.5, 0.333, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.667, 0.333, 0.333, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 0.0, 0.333, 0.333, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.667, 0.333, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.455, 0.667, 0.333, 0.667, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.286, 0.286, 0.0, 0.0, 1.0, 0.0, 0.333, 1.0, 0.667, 0.778, 0.556, 0.0, 0.667, 0.667, 1.0, 0.333, 0.0, 0.667, 0.333, 0.667, 0.0, 0.182, 0.333, 0.333, 1.0, 1.0, 0.333, 0.333, 0.7, 0.0, 0.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.25, 0.333, 0.625, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.667, 0.0, 0.333, 0.0, 1.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.667, 0.333, 0.667, 0.5, 0.5, 0.0, 0.25, 0.0, 0.0, 0.5, 0.0, 0.0, 0.7, 0.0, 0.25, 0.0, 0.333, 0.0, 0.667, 0.0, 0.0, 0.333, 0.0, 0.667, 0.333, 0.0, 0.667, 0.667, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 1.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.25, 0.0, 0.0, 0.333, 0.0, 0.333, 0.667, 1.5, 0.5, 1.333, 0.5, 0.0, 0.667, 1.0, 0.333, 0.667, 0.0, 0.0, 0.5, 0.0, 0.333, 0.333, 0.5, 0.333, 1.0, 1.0, 0.333, 0.333, 0.667, 0.333, 0.667, 0.0, 0.75, 0.5, 0.0, 0.0, 0.333, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.667, 0.333, 0.333, 0.0, 0.667, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.667, 1.0, 0.5, 0.5, 0.25, 1.0, 0.0, 0.333, 0.667, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.0, 1.125, 0.875, 1.0, 0.0, 0.0, 0.75, 0.75, 1.0, 0.5, 0.0, 0.25, 0.0, 0.667, 0.0, 0.667, 0.667, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.333, 0.0, 0.0, 0.25, 0.25, 0.0, 0.667, 0.0, 0.0, 0.667, 0.0, 1.0, 1.0, 0.333, 0.0, 0.0, 0.667, 0.0, 0.667, 0.667, 0.0, 0.0, 0.667, 0.333, 1.0, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667, 0.0, 0.667, 0.0, 0.667, 0.667, 0.333, 0.0, 0.333, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.25, 0.333, 0.333, 0.0, 1.5, 0.0, 0.667, 0.333, 0.333, 0.333, 0.0, 0.0, 0.0, 0.667, 1.0, 0.0, 0.0, 0.5, 1.0, 0.5, 0.0, 0.333, 0.0, 0.0, 1.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.333, 0.667, 0.0, 0.0, 0.333, 0.0, 0.0, 0.0, 1.0, 1.0, 0.667, 0.667, 0.667, 1.0, 0.333, 1.333, 1.0, 0.333, 0.0, 0.333, 0.333, 0.333, 0.333, 0.0, 1.5, 1.0]



In [5]:
print("For ne_np total no of WER values:",len(ne_np_female))
print("For merged all total no of WER values:",len(merged_all))
x=ne_np_female
y=merged_all
# Avg_WER=sum(weer_list)/len(weer_list)
# print(Avg_WER)

For ne_np total no of WER values: 210
For merged all total no of WER values: 11027


In [6]:
Avg_WER_female=sum(x)/len(x)
Avg_WER_merged=sum(y)/len(y)
print("AvgWer_female:",Avg_WER_female)
print("AvgWer_merged:",Avg_WER_merged)

AvgWer_female: 0.5375714285714288
AvgWer_merged: 0.412978597986759


In [16]:
x=[1,2,3,4,5,6,7]
y=[]
for i in range(2,5):
    y.append(i)
z=x[2:5]

In [17]:
print(y)
print(z)

[2, 3, 4]
[3, 4, 5]


In [22]:
shi="myname:1213"
x=shi.split(":")
print(x[0])
print("\n")
print(x[1])

myname


1213
